## Análise técnica baseado em Médias Móveis

- Análise técnica estuda os movimentos do mercado por meio de gráficos baseado no preço das cotações dos ativos
- Uma média móvel é a média do preço de um ativo considerado um periodo passado de tempo 
    - pode fornecer informações que vão lhe auxiliar no processo de decisão relacionada a investimentos
    - pode indicar a "força" do mercado
    - pode fornecer insights sobre o momento de entrar ou sair de um determinado ativo
- Valores de suporte e valores de resistências

<img src="suporte-resistencia.png" width="600"/>

In [1]:
import pandas as pd
import pandas_datareader as data
import plotly.graph_objs as go

ModuleNotFoundError: No module named 'pandas_datareader'

## Leitura de dados

In [ ]:
start_date = '2010-01-01'
end_date = '2021-01-01'

In [ ]:
ibov = data.DataReader('^BVSP', 'yahoo', start_date, end_date)

In [ ]:
ibov

## Cálculando Rentabilidade

- 100 * preço atual / preço anterior - 100
- 100 * (preco atual - preco anterior) / preco anterior

In [ ]:
ibov['rentab'] = ibov['Close'] / ibov['Close'].shift() * 100 -100

In [ ]:
ibov['rentab_b'] = (ibov['Close'] - ibov['Close'].shift()) / ibov['Close'].shift() * 100 

In [ ]:
ibov

## Cálculando Médias Móveis

In [ ]:
ibov['mv_7d'] = ibov['Close'].rolling(7).mean()
ibov['mv_21d'] = ibov['Close'].rolling(21).mean()
ibov['mv_50d'] = ibov['Close'].rolling(50).mean()
ibov['mv_200d'] = ibov['Close'].rolling(200).mean()

## Plotando os dados
- https://plotly.com/python/candlestick-charts/

In [ ]:
def plotCandleStick(df_, name='ticket', lines=[]):
    trace1 = {
        'x': df_.index,
        'open': df_.Open,
        'close': df_.Close,
        'high': df_.High,
        'low': df_.Low,
        'type': 'candlestick',
        'name': name,
        'showlegend': False
    }

    data = [trace1]
    layout = go.Layout()

    fig = go.Figure(data=data, layout=layout)
    
    if len(lines) > 0:
        for c in lines:
            fig.add_trace(
                go.Scatter(x = list(df_.index),
                           y = df_[c],
                           mode = 'lines',
                           name = c))     
        
    return fig

In [ ]:
plotCandleStick(ibov, name='Ibovespa', lines=['mv_7d', 'mv_21d','mv_50d', 'mv_200d'])

In [ ]:
# plot_stock(ibov, columns=['Close', 'mv_7d', 'mv_21d', 'mv_50d', 'mv_200d'])

In [ ]:
ibov[ibov['mv_7d'] > ibov['mv_200d']]

### Extrair features do datetime

In [ ]:
filter_entrada = ibov['mv_7d'] > ibov['mv_200d']

In [ ]:
def features_extraction(df_):
    df_.reset_index(inplace=True)
    df_['year'] = df_['Date'].dt.year
    df_['month'] = df_['Date'].dt.month
    df_['day'] = df_['Date'].dt.day

In [ ]:
features_extraction(ibov)
ibov.set_index('Date', inplace=True)

In [ ]:
ibov

In [ ]:
ibov[filter_entrada].groupby('month').agg({'month':'count'}).plot(kind='bar')

In [ ]:
ibov[filter_entrada].groupby('day').agg({'day':'count'}).plot(kind='bar')

## Conclusões
- Dados de médias móveis para auxiliar o seu processo de tomada de decisão
- Tenha uma estratégia para tomar decisões orientadas a dados